# Introduction
I have decided to deal with the relationship between guns captured by the police in the city of Rio de Janeiro and the amount of thefts, robberies, threats and reports to the police, as Rio de Janeiro is know for being one og the most violent cities in the world. 

In my opinion many might be interested in the problem but most likely people involved with security of citizens in Rio de Janeiro, so police and politicians mostly.

# Data
The data I used was taken from the public security portal here <a href="http://www.ispdados.rj.gov.br/estatistica.html">Stats</a>. The files I used were the top three, those are monthly statistics for almost all things public security related and then statistics for guns apprehended. All statistics I got went from 2003 all through 2019. 

In [17]:
# File with all data public security related
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dfCisp = pd.read_csv('http://www.ispdados.rj.gov.br/Arquivos/BaseDPEvolucaoMensalCisp.csv', encoding = "ISO-8859-1", delimiter = ';')
dfCisp.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CISP,mes,vano,mes_ano,AISP,RISP,munic,mcirc,Regiao,hom_doloso,...,pessoas_desaparecidas,encontro_cadaver,encontro_ossada,pol_militares_mortos_serv,pol_civis_mortos_serv,indicador_letalidade,indicador_roubo_rua,indicador_roubo_veic,registro_ocorrencias,fase
0,1,1,2003,2003m1,5,1,Rio de Janeiro,3304557,Capital,0,...,2,0,0,0,0,0,66,5,578,3
1,4,1,2003,2003m1,5,1,Rio de Janeiro,3304557,Capital,3,...,6,0,1,0,0,3,51,9,441,3
2,5,1,2003,2003m1,5,1,Rio de Janeiro,3304557,Capital,3,...,2,1,0,0,0,3,64,11,637,3
3,6,1,2003,2003m1,1,1,Rio de Janeiro,3304557,Capital,6,...,2,1,0,0,0,6,56,27,473,3
4,7,1,2003,2003m1,1,1,Rio de Janeiro,3304557,Capital,4,...,1,3,0,0,0,4,5,23,147,3


In [3]:
# Guns from 2000-2006
dfA1 = pd.read_csv('http://www.ispdados.rj.gov.br/Arquivos/ArmasEstado2000_2006.csv', encoding = "ISO-8859-1", delimiter = ';')
dfA1.head()

,vano,mes,armas_apreendidas
0,2000,1,697
1,2000,2,751
2,2000,3,784
3,2000,4,780
4,2000,5,761


In [4]:
# Guns from 2006-2019 (categorized)
dfGuns = pd.read_excel('http://www.ispdados.rj.gov.br/Arquivos/ArmasApreendidasEvolucaoCisp.xlsx', index_col = 0, sheet_name = "Arma_de_Fogo")
dfGuns.head()

,mes,vano,aisp,risp,arma_fabricacao_caseira,carabina,espingarda,fuzil,garrucha,garruchao,metralhadora,outros,pistola,revolver,submetralhadora,total
circ,,,,,,,,,,,,,,,,
1,1,2007,5,1,0,0,0,0,0,0,0,0,0,2,0,2
1,2,2007,5,1,0,0,0,0,0,0,0,0,0,1,0,1
1,3,2007,5,1,0,0,0,0,0,0,0,0,0,0,0,0
1,4,2007,5,1,0,0,0,0,0,0,0,1,4,1,0,6
1,5,2007,5,1,0,0,0,0,6,0,0,0,1,2,0,9


The data was cleaned up and most of the columns left out. I only used total number of guns to have the guns apprehended databases the same. In all stats I removed most of the columns, only keeping the total number of thefts, robberies, threats and reports.

In [13]:
# Clean unnecessary columns
dfA2 = dfGuns.drop(['aisp', 'risp', 'arma_fabricacao_caseira', 'carabina',
       'espingarda', 'fuzil', 'garrucha', 'garruchao', 'metralhadora',
       'outros', 'pistola', 'revolver', 'submetralhadora'], axis=1)

# Clean the database and set up total number of guns
dfA1.columns = ['Year', 'Month', 'Guns']
dfA2.columns = ['Month', 'Year', 'Guns']
dfA2 = dfA2.groupby(['Year','Month']).sum()
dfA2.reset_index(inplace = True)

# Join the guns databases
dfA = dfA1.append(dfA2)
dfA.head()

,Year,Month,Guns
0,2000,1,697
1,2000,2,751
2,2000,3,784
3,2000,4,780
4,2000,5,761


In [14]:
# Drop unnecessary columns
dfC = dfCisp.drop(['CISP', 'mes_ano', 'munic', 'AISP', 'RISP', 'mcirc', 'cmp', 'cmba', 'aaapai', 'apf', 'roubo_comercio', 'roubo_residencia',
       'roubo_veiculo', 'roubo_carga', 'roubo_transeunte', 'roubo_em_coletivo',
       'roubo_banco', 'roubo_cx_eletronico', 'roubo_celular',
       'roubo_conducao_saque', 'roubo_apos_saque', 'roubo_bicicleta',
       'outros_roubos', 'furto_veiculos', 'furto_transeunte',
       'furto_coletivo', 'furto_celular', 'furto_bicicleta', 'outros_furtos', 
        'sequestro', 'extorsao', 'sequestro_relampago', 'estelionato', 'apreensao_drogas',
       'posse_drogas', 'trafico_drogas', 'apreensao_drogas_sem_autor',
       'recuperacao_veiculos', 'fase', 'pessoas_desaparecidas',
       'encontro_cadaver', 'encontro_ossada', 'pol_militares_mortos_serv',
       'pol_civis_mortos_serv', 'hom_doloso', 'lesao_corp_morte',
       'latrocinio', 'hom_por_interv_policial', 'tentat_hom',
       'lesao_corp_dolosa', 'estupro', 'hom_culposo', 'lesao_corp_culposa',
        'indicador_letalidade', 'indicador_roubo_rua', 'indicador_roubo_veic', 'Regiao'], axis=1)


#Rename columns
dfC.columns = ['Month', 'Year', 'Robberies', 'Thefts', 'Threats',
       'Reports']

# Group together stats from different areas
dfC = dfC.groupby(['Year', 'Month']).sum()
dfC.reset_index(inplace = True)

dfC.head()

,Year,Month,Robberies,Thefts,Threats,Reports
0,2003,1,10296,10483,6127,44793
1,2003,2,10634,10335,6047,45403
2,2003,3,9976,10976,5793,45417
3,2003,4,10222,10409,5626,44184
4,2003,5,10875,10414,5436,45572


In [15]:
# Join the databases for final database for analysis
df = pd.merge(dfC, dfA, on=['Year', 'Month'])
df.head()

,Year,Month,Robberies,Thefts,Threats,Reports,Guns
0,2003,1,10296,10483,6127,44793,1154
1,2003,2,10634,10335,6047,45403,1324
2,2003,3,9976,10976,5793,45417,1344
3,2003,4,10222,10409,5626,44184,1336
4,2003,5,10875,10414,5436,45572,1292
